In [12]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler
import itertools
from torch import optim 
import random 
import os 
from PIL import Image 
import torch.utils.data as data
import os.path
from torch.utils.data import Dataset
from torchvision import datasets,transforms,utils
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
import numpy as np
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
from IPython import display
import numpy
import random
from tkinter import *
from tkinter.messagebox import showinfo
import pdb
from tkinter import filedialog
from PIL import Image, ImageTk

In [13]:
# Class that defines the structure of the dataset
# Sets up the file paths to each style domain image folder
# Transforms and prepares each image for enumeration
# Creates an indexing system for the dataset 

class DatasetStructure():
    
    def initialize(self,path,loadSize,fineSize,color):
        
        self.dir_A = os.path.join(path, 'train' + 'A')
        self.dir_B = os.path.join(path, 'train' + 'B')
        
        self.A_paths = []
    
        for root, _, files in sorted(os.walk(self.dir_A)):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    path = os.path.join(root, file)
                    self.A_paths.append(path)
                    
        self.B_paths = []
    
        for root, _, files in sorted(os.walk(self.dir_B)):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    path = os.path.join(root, file)
                    self.B_paths.append(path)                 

        self.A_paths = sorted(self.A_paths)
        self.B_paths = sorted(self.B_paths)
        
        self.A_size = len(self.A_paths)
        self.B_size = len(self.B_paths)
        
        self.transform = get_transform(loadSize,fineSize,color) 
        
    def __getitem__(self, index):
        
        A_path = self.A_paths[index % self.A_size]

        index_B = index % self.B_size

        B_path = self.B_paths[index_B]
        A_img = Image.open(A_path).convert('RGB')
        B_img = Image.open(B_path).convert('RGB')

        A = self.transform(A_img)
        B = self.transform(B_img)

        return {'A': A, 'B': B,
                'A_paths': A_path, 'B_paths': B_path}

    def __len__(self):
        return max(self.A_size, self.B_size)

# Data set creation functions

# Function that creates a list of transforms from the Pytorch library
# Each image in the dataset is passed through the transforms
# Transforms include: Transfer to Tensor data structure, random crops, and normalization

def get_transform(load_size,crop_size,color):
    
    transform_list = [transforms.Resize([load_size,load_size],Image.BICUBIC),transforms.RandomCrop(crop_size),
                      transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
    
    if color == 'grayscale':
        transform_list.append(transforms.Grayscale(num_output_channels = 3))
        
    composed_transforms = transforms.Compose(transform_list)
    
    return composed_transforms  
    
# Creates an instance of the dataset
# Specifies the image sizes in the dataset and whether they are gray or color

def createDataset(path,load_size,crop_size,color):
    
    instance = DatasetStructure()
    instance.initialize(path,load_size,crop_size,color)
    
    dataset = instance
    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=not 'store_true', num_workers=int(0))
    
    return dataloader

# Function that gets a chosen number of images from the dataset in order to display the network training progress

def get_test_dataset(num_imgs,dataset):
    
    a = 0
    tensor_list_A = []
 
    for i, data in enumerate(dataset): 
        real_A = data['A']
        tensor_list_A.append(real_A[0])
       
        a +=1
        if a == num_imgs:
            break
        
    img_list = torch.stack(tensor_list_A) 
    return img_list

In [14]:
# Network functions
    
class Discriminator(nn.Module):
    def __init__(self, norm_layer=nn.BatchNorm2d, use_sigmoid=False):
        super(Discriminator, self).__init__()
        
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        
        NN_sequence = [nn.Conv2d(3,64,kernel_size=4,stride=2,padding=1),nn.LeakyReLU(0.2,True),
                    
                    nn.Conv2d(64, 64 * 2,kernel_size=4, stride=2, padding=1, bias=use_bias),
                    norm_layer(64 * 2),nn.LeakyReLU(0.2, True),
                    
                    nn.Conv2d(64 * 2, 64 * 4,kernel_size=4, stride=2, padding=1, bias=use_bias),
                    norm_layer(64 * 4),nn.LeakyReLU(0.2, True),
                    
                    nn.Conv2d(64 * 4, 64 * 8,kernel_size=4, stride=2, padding=1, bias=use_bias),
                    norm_layer(64 * 8),nn.LeakyReLU(0.2, True),
                    
                    nn.Conv2d(64 * 8, 64 * 8,kernel_size=4, stride=1, padding=1, bias=use_bias),
                    norm_layer(64 * 8),nn.LeakyReLU(0.2, True),
                    
                    nn.Conv2d(64 * 8, 1, kernel_size=4, stride=1, padding=1)]

        if use_sigmoid:
            NN_sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*NN_sequence)
        
    def forward(self, input):
        return self.model(input) 
    
class Generator(nn.Module):
    def __init__(self,norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=9, padding_type='reflect'):
        assert(n_blocks >= 0)
        super(Generator, self).__init__()
        
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d 
            
        encoder = [nn.ReflectionPad2d(3),nn.Conv2d(3, 64, kernel_size=7, padding=0,bias=use_bias),norm_layer(64),nn.ReLU(True), 
                 
                 nn.Conv2d(64, 64 * 2, kernel_size=3,stride=2, padding=1, bias=use_bias),
                 norm_layer(64 * 2),nn.ReLU(True),
                 
                 nn.Conv2d(64 * 2, 64 * 4, kernel_size=3,stride=2, padding=1, bias=use_bias),
                 norm_layer(64 * 4),nn.ReLU(True)]
        
        transformer = []
        for i in range(n_blocks):
            transformer += [ResnetBlock(64 * 4, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, 
                                  use_bias=use_bias)]           
       
        decoder = [nn.ConvTranspose2d(64 * 4, int(64 * 4 / 2),kernel_size=3, stride=2,padding=1, output_padding=1,
                                      bias=use_bias),norm_layer(int(64 * 4 / 2)),nn.ReLU(True), 
                   
                   nn.ConvTranspose2d(64 * 2, int(64 * 2 / 2),kernel_size=3, stride=2,padding=1, output_padding=1,
                                      bias=use_bias),norm_layer(int(64 * 2 / 2)),nn.ReLU(True),
                   
                   nn.ReflectionPad2d(3),
                   
                   nn.Conv2d(64, 3, kernel_size=7, padding=0),
                   
                   nn.Tanh()]

        model = encoder + transformer + decoder
        
        self.model = nn.Sequential(*model)
        
    def forward(self, input):
        return self.model(input)
    
# Define a resnet block
class ResnetBlock(nn.Module):
    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        super(ResnetBlock, self).__init__()
        
        self.block = self.build_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):

        block = [nn.ReflectionPad2d(1),
                       
                 nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=use_bias),norm_layer(dim),nn.ReLU(True)]
        
        if use_dropout:
            block += [nn.Dropout(0.5)]

        block += [nn.ReflectionPad2d(1),
                       
                  nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=use_bias),norm_layer(dim)]

        return nn.Sequential(*block)

    def forward(self, x):
        out = x + self.block(x)
        return out
    
# initializes NN weights based on their classname
# Convolution and Linear classes given weights from normalized distribution with mean 0.0 and standard deviation 0.03
# Batch norm class given weights from normalized distribution with mean 0.9 and standard deviation 0.03
# Bias weights are set at a constant 0

def normal_weight_init(NN):
    def weight_assign(m):
        classname = m.__class__.__name__       
        # Find the weights of the convolution and linear classes and assign a normal weight distribution        
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, 0.03)            
            # Set bias to 0.0           
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:
            init.normal_(m.weight.data, 0.9, 0.03)
            init.constant_(m.bias.data, 0.0)            
    NN.apply(weight_assign)  

# Creates all of the models required for CycleGAN training
# Includes Discriminators for each domains (DA and DB)
# as well as the Generator transforming domain A into domain B
# and the generator transforming domain B into domain A
    
def createModels():
    norm_layer = functools.partial(nn.BatchNorm2d, affine=True)
    
    DA = Discriminator(norm_layer, 'store_true')
    DA.to('cuda')  
    normal_weight_init(DA)
    
    DB = Discriminator(norm_layer, 'store_true')
    DB.to('cuda')  
    normal_weight_init(DB)
    
    GAB = Generator(norm_layer, not 'store_true', n_blocks=9)
    GAB.to('cuda')  
    normal_weight_init(GAB)
    
    GBA = Generator(norm_layer, not 'store_true', n_blocks=9)
    GBA.to('cuda')  
    normal_weight_init(GBA)
        
    return GAB,GBA,DA,DB

def createNN(NN_type,gpu):
    
    norm_layer = functools.partial(nn.BatchNorm2d, affine=True)
    
    if NN_type == 'generator':
        NN = Generator(norm_layer, not 'store_true', n_blocks=9)

    elif NN_type == 'discriminator':
        NN = Discriminator(norm_layer, 'store_true')
      
    if gpu:
        NN.to('cuda')  
    else:
        NN.to('cpu')  
        
    normal_weight_init(NN)
        
    return NN

In [15]:
# Training functions

def update_pool(imgs,pool):
    pool_imgs = []
    for img in imgs:
        img = torch.unsqueeze(img.data, 0)
        if pool[1] < pool[0]:
            pool[1] = pool[1] + 1
            pool[2].append(img)
            pool_imgs.append(img)
        else:
            if random.uniform(0, 1) > 0.5:
                r_id = random.randint(0, pool[0] - 1)
                temp_img = pool[2][r_id].clone()
                pool[2][r_id] = img
                pool_imgs.append(temp_img)
            else:
                pool_imgs.append(img)
    pool_imgs = torch.cat(pool_imgs, 0)
    return pool_imgs

def gan_loss(input,target):
    
    loss = nn.BCELoss()
        
    if target:
        target_tensor = torch.tensor(1.0)
    else:
        target_tensor = torch.tensor(0.0)
        
    target_tensor = target_tensor.expand_as(input)
    target_tensor = target_tensor.cuda()
        
    return loss(input,target_tensor)

# learning rate functions

def get_scheduler(optimizer):

    def lambda_rule(epoch):
        lr_l = 1.0 - max(0, epoch - 99) / float(101)
        return lr_l
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)

    return scheduler
        
def update_learning_rate(schedulers,optimizers):
    for scheduler in schedulers:
        scheduler.step()
    lr = optimizers[0].param_groups[0]['lr']
    
def createOptimizers():
      
    optimizer_G = torch.optim.Adam(itertools.chain(GAB.parameters(), GBA.parameters()),0.0002, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(itertools.chain(DA.parameters(), DB.parameters()),0.0002, betas=(0.5, 0.999))
    optimizers = [optimizer_G,optimizer_D]
    
    return optimizers

def optimize_parameters(real_A,real_B,optimizers):
    
    lf = torch.nn.L1Loss()
    
    fake_B = GAB(real_A)
    re_A = GBA(fake_B)
    fake_A = GBA(real_B)
    re_B = GAB(fake_A)
    
    nets = [DA, DB]
    requires_grad = False    
    for net in nets:
        for param in net.parameters():
            param.requires_grad = requires_grad           
    optimizers[0].zero_grad()
    
    g_loss = (gan_loss(DA(fake_B), True)) + (gan_loss(DB(fake_A), True)) + (lf(re_A, real_A) * 10.0) + \
             (lf(re_B, real_B) * 10.0) + (lf(GAB(real_B), real_B) * 10.0 * 0.5) + \
             (lf(GBA(real_A), real_A) * 10.0 * 0.5)
    
    g_loss.backward()
    
    optimizers[0].step() 
        
    nets = [DA, DB]
    requires_grad = True   
    for net in nets:
        for param in net.parameters():
            param.requires_grad = requires_grad            
    optimizers[1].zero_grad()  
    
    # calculate discrimintor A loss, by predicting real images and generator AB fake images
     
    fake_B = update_pool(fake_B,B_pool)    
    pred_real_DA = DA(real_B)
    pred_fake_DA = DA(fake_B.detach())       
    DA_loss = ((gan_loss(pred_real_DA, True)) + (gan_loss(pred_fake_DA, False))) * 0.5
    DA_loss.backward() 
    
    # calculate discrimintor B loss, by predicting real images and generator AB fake images    
    
    fake_A = update_pool(fake_A,A_pool)   
    pred_real_DB = DB(real_A)    
    pred_fake_DB = DB(fake_A.detach())    
    DB_loss = ((gan_loss(pred_real_DB, True)) + (gan_loss(pred_fake_DB, False))) * 0.5
    DB_loss.backward()
    
    optimizers[1].step()  
    
def startTraining(epoch_limit,in_size,out_size,sample_num,path,save_image_path,schedulers,optimizers):

    # create dataset

    dataset = createDataset(train_path,in_size,out_size,'color')
    img_list = get_test_dataset(sample_num,dataset)
        
    # training process

    for epoch in range(1,epoch_limit):
        i = 0
        for i, data in enumerate(dataset):
        
            real_A = data['A']
            real_B = data['B']
            if torch.cuda.is_available(): real_A = real_A.cuda()
            if torch.cuda.is_available(): real_B = real_B.cuda()
  
            optimize_parameters(real_A,real_B,optimizers)
        
            # Display Progress
            if (i) % 100 == 0:
                display.clear_output(True)
                # Display Images
                test_images_B = GAB(img_list.cuda()).data.cpu()
                        
                saveImages(save_image_path,test_images_B,epoch,i,'cyclegan','test_save',i)

                print (i)
                print ('epoch: ' ,epoch,'/',200)     

                saveModel(GAB,epoch,str(i),path)
            i = i + 1
        
        update_learning_rate(schedulers,optimizers)
        

In [16]:
# Logger functions

# Save trained models and images functions

def saveModel(NN,epoch,batch,path):
    
    torch.save(NN.state_dict(),'{}/NN_epoch_{}_{}'.format(path, epoch,batch))

def saveImages(save_image_path,images,epoch,n_batch,model_name,data_name,num_batches):
    
    # Make horizontal grid from image tensor
    horizontal_grid = vutils.make_grid(images, True, scale_each=True)
    
    # Add horizontal images to tensorboard
    step = epoch * num_batches + n_batch
    comment = '{}_{}'.format(model_name, data_name)
    img_name = '{}/images{}'.format(comment, '')
    writer = SummaryWriter(comment)
    writer.add_image(img_name, horizontal_grid, step)
    
    # Plot and save horizontal
    fig = plt.figure(figsize=(16, 16))
    plt.imshow(np.moveaxis(horizontal_grid.numpy(), 0, -1))
    plt.axis('off')
    display.display(plt.gcf())
        
    fig.savefig('{}/epoch_{}_batch_{}.png'.format(save_image_path,epoch, n_batch))
    plt.close()
    
# function that saves image list to a file

def saveImgList(img_list,save_path2,name):
    for i in range(0,len(img_list)):
        utils.save_image(img_list[i], save_path2 + name +'img.jpg',normalize = True)  

In [17]:
# Create Classifier Functions

def optimize_classifier(classA,classB,D,optimizer_D):
  
    nets = [D]
    requires_grad = True    
    for net in nets:
        for param in net.parameters():
            param.requires_grad = requires_grad                                
    optimizer_D.zero_grad()
    
    pred_real_D = D(classA)
    pred_fake_D = D(classB)    
    D_loss = ((gan_loss(pred_real_D, True)) + (gan_loss(pred_fake_D, False))) * 0.5   
    D_loss.backward()
        
    optimizer_D.step()
    
def createClassifier(epoch_limit,data_path,save_classifier_path,size,color):
    
    D = createNN('discriminator',True)
    opt_D = torch.optim.Adam(itertools.chain(D.parameters()),0.0002, betas=(0.5, 0.999))
    schedulers = [get_scheduler(opt_D)]
    
    dataset = createDataset(data_path,size,size,color)

    epoch = 1
    for epoch in range(epoch, epoch_limit):
        i=0
        for i, data in enumerate(dataset):
            real_A = data['A']
            real_B = data['B']
     
            if torch.cuda.is_available(): real_A = real_A.cuda()
            if torch.cuda.is_available(): real_B = real_B.cuda()
  
            optimize_classifier(real_A,real_B,D,opt_D)
        
            # Display Progress
            if (i) % 100 == 0:
                display.clear_output(True)
                print (i)
                print ('epoch: ' ,epoch,'/',epoch_limit) 
                            
            i = i + 1
            
        saveModel(D,epoch,'cls',save_classifier_path)
            
        update_learning_rate(schedulers,[opt_D])  

In [ ]:
# ---------------------------------
# Genetic Algorithm Functions
# ---------------------------------

In [18]:
# Helper function that uploads and prepares a batch of images from a specified path
# Parameters decide the size of the images and whether they are color or gray

def get_images(path,size,color):
    
    test_dataset = createDataset(path,size,size,color)

    tensor_list = []    
    for i, data in enumerate(test_dataset):
        real_A = data['A']
        real_B = data['B']
        real = [real_A,real_B]
        tensor_list.append(real[0][0])     
 
    img_data = torch.stack(tensor_list)   
    return img_data

# String together multiple random NNs into a chain by placing them into a list 

def create_NN_chain(chain_size, noiseNN_limit,colorNN_limit):
    
    NN_chain = []
    chain_num = ''
    
    for i in range(1,chain_size):
        
        # Randomly select NN index
        
        colorNN_choice = (random.randint(1,colorNN_limit))
        noiseNN_choice = (random.randint(1,noiseNN_limit))
        desc = ''
        
        # Only first NN of the chain converts noise to color image
        
        if i == 1:
            model_path = n2c_NN_path + '(' + str(noiseNN_choice) + ')'
            desc = str(noiseNN_choice)            
        else:            
            model_path = c2r_NN_path + '(' + str(colorNN_choice) + ')'
            desc = str(colorNN_choice)
         
        # Instantiate selected NN taken from the chosen path and append it to the chain
        
        NN = createNN('generator',True)
        NN.load_state_dict(torch.load(model_path,map_location = 'cuda'))       
        NN_chain.append(NN)        
        chain_num = chain_num + '-'+ desc
        
    return NN_chain,chain_num

# Functions that pass data through the NN chain and return resulting images
def NNPass(noise_data, NN_chain,chain_path):
    
    score_image_list = []
    
    for img in noise_data:

        for i in range(0,len(NN_chain)):
            if i == len(NN_chain):
                end = True 
            else:
                end = False
                
            img = chainPass(NN_chain,i,img,chain_path,end)
            
        score_image_list.append(img)
        
    return score_image_list

def chainPass(NN_chain,chain_idx,img,chain_path,end):
    img1 = torch.tensor(img)
    img2 = torch.stack((img1,img1))      
    img3 = NN_chain[chain_idx](img2.cuda()).data.cpu()
    img4 = img3[0]
               
    utils.save_image(img3[0], chain_path+'img1.jpg',normalize = True)
               
    nd = get_images(reupload_path,256,'color')
               
    img5 = nd[0]
    
    if end:
        return img4
    else:
        return img5

# function that scores a list of images with the discriminator fitness function

def scoreImgList(path,classifier,color):
    
    img_list = get_images(path,256,color)
    
    img_scores =[]
    for i in img_list:
        img = torch.tensor(i)
        img2 = torch.stack((img,img))
        img_score = classifier(img2.cuda()).data.cpu()
        img_scores.append((torch.mean(img_score).numpy())*-1)
        
        #print((torch.mean(img_score).numpy()))

    # avg is the score of the NN chain 

    img_score_avg = sum(img_scores)/len(img_scores)
        
    return(img_score_avg) 

# function that creates a population of NN chains

def createPopulation(noise_data,save_path,population_size,chain_size):

    pop = []
    scores = []
    pop_desc = []
    
    for i in range(0,population_size):
        
        NN_chain,chain_num = create_NN_chain(chain_size,5,5)    
        pop.append(NN_chain)
    
        new_img_list = NNPass(noise_data,NN_chain,chain_path)

        # save out images from NN chain

        saveImgList(new_img_list,save_path,'') 
        
        # Uses classifier to get score of each image in the img_list created by the NN chain

        NN_chain_score = scoreImgList(score_path,classifier,'grayscale')
        
        scores.append(NN_chain_score)
        pop_desc.append(chain_num)
        
    return pop,scores,pop_desc

# function to pick two members of the population and compare their scores, outputs winner and loser

def getWinnerLoser(popul,scores,kd):
    population_size = len(popul)

    # Choose an index1 from 0 to 9 and an index2 a certain distance from index1

    id1 = random.randint(0,population_size-1)
    d = random.randint(0,kd)
    id2 = id1 + d
    if id2 > population_size-1:
        id2 = id2 - population_size-1
            
    # compare scores for the NN chains
    # Put winner NN chain into W and loser NN chain into L
   
    if scores[id1] >= scores[id2]:
        W = popul[id1]
        L = popul[id2]
        w_id = id1
        l_id = id2


    else:
        W = popul[id2]
        L = popul[id1]
        w_id = id2
        l_id = id1

            
    return(W,L,w_id,l_id)

# cross the 'genetics' of the winner with the loser, randomly mutate the loser

def mutate_loser(W,L,cross,mutate,l_id):
    
    # Switch out NNs in the loser's array for NNs in the winners array

    for i in range(0,len(L)):
        x = random.randint(0,100)
        if x < cross:
            L[i] = W[i]
        #if x < mutate and l_id != scores.index(max(scores)):
        if x < mutate:
            if i == 0:
                rand = (random.randint(1,5))
                model_path = n2c_NN_path + '(' + str(rand) + ')'    
                NNN = createNN('generator',True)
                NNN.load_state_dict(torch.load(model_path,map_location = 'cuda'))                   
                L[i] = NNN
            else:
                rand = (random.randint(1,5))
                model_path = c2r_NN_path + '(' + str(rand) + ')'    
                NNN = createNN('generator',True)
                NNN.load_state_dict(torch.load(model_path,map_location = 'cuda'))                   
                L[i] = NNN 
                
    return L

# start genetic algorithm loop

def geneticAlgorithm(epochs,pop,scores,noise_data,chain_path,save_path,score_path,classifier):
    
    best_score = -100
    kd = 1
    
    scorez = scores
    best_chain_scores = []
    best_chains = []
    popu = pop
    for i in range(1,epochs):
        W,L,w_id,l_id = getWinnerLoser(popu,scorez,kd)

    
        mutated_L = mutate_loser(W,L,60,20,l_id)
        popu[l_id] = mutated_L
    
        mutated_L_img_list = NNPass(noise_data, popu[l_id],chain_path)
    
        saveImgList(mutated_L_img_list,save_path,'')
        
    
        mutated_L_score = scoreImgList(score_path,classifier,'grayscale')
               
        scorez[l_id] = mutated_L_score
        
        r = random.randint(0,100)
        if kd < len(L)-1 and r < 9:
            kd = kd+1
            
        display.clear_output(True)    
        print('epoch',str(i),'-',max(scorez))
            
        best_score = max(scorez)
    
    return popu,best_chains,best_chain_scores

In [19]:
# Multiple Network render function

def multiNNRender(data_path,model_path,save_path,first,last,size):
    
    device_choice = 'cuda'
      
    dataset = createDataset(data_path,size,size,'color')

    img_list = get_test_dataset(4,dataset)
    
    for i in range(first,last):
        NN_path = model_path + str(i) + ')'
        G = createNN('generator',False)
        G.load_state_dict(torch.load(NN_path,map_location = device_choice))        
        new_imgs =[]
        
        for j in img_list:
            img = torch.tensor(j)
            img2 = torch.stack((img,img))
            img3 = G(img2.cuda()).data.cpu()
            new_imgs.append(img3[0])      
            
        saveImgList(new_imgs,save_path,str(i))

In [20]:
# App functions
def useGenerator(img_data,model_path,save_path,name,gpu):
    
    new_imgs =[]   
    img = torch.tensor(img_data[0])
    img2 = torch.stack((img,img))  
    
    img3 = G2(img2)
    
    new_imgs.append(img3[0])

    saveImgList(new_imgs,save_path,'')
    
def NNPath():
    global NN_path
    NN_path = filedialog.askopenfilename()

    G2.load_state_dict(torch.load(NN_path,map_location = device))
    w.create_rectangle(265,10,290,35, fill='lightgreen')
    
def imagePath():
    global image_path
    image_path = filedialog.askopenfilename()
    
    if image_path != '':
        load = Image.open(image_path)
        load = load.resize((260,260))
        render = ImageTk.PhotoImage(load)
        l = Label(image=render)
        l.photo = render

        l.place(x=19,y=59)
        
def createNewImage():
    
    if image_path is not '' and NN_path is not '':
    
        A_img = Image.open(image_path).convert('RGB')
        
        A = transform(A_img)
        A1 = torch.stack((A,A))

        useGenerator(A1,NN_path,save_path,'ok',False)
        
        load2 = Image.open(new_path)
        load2 = load2.resize((260,260))
        render2 = ImageTk.PhotoImage(load2)
        l2 = Label(image=render2)
        l2.photo = render2

        l2.place(x=359,y=59)
        
def app(gpu):
        
    norm_layer = functools.partial(nn.BatchNorm2d, affine=True)   
    use_dropout = not 'store_true'
    G2 = Generator(norm_layer, use_dropout, n_blocks=9)
    
    if gpu:
        device = torch.device('cuda')
        G2.to('cuda')
    else:
        device = torch.device('cpu')
        
    normal_weight_init(G2)    
    main_path = os.path.dirname(os.path.abspath('__file__'))
    save_path = main_path+'/saved_images/save/1'
    transform = get_transform(256,256,'color')    
    new_path = main_path+'/saved_images/save/1img.jpg'
    
    return G2,device,transform,save_path,new_path     

In [21]:
run_choice = 'app'

In [22]:
if run_choice == 'train':
    
    # Start training process

    GAB,GBA,DA,DB = createModels()

    optimizers = createOptimizers()

    schedulers = [get_scheduler(optimizer) for optimizer in optimizers]

    pool_size = 50
    A_num_imgs = 0
    A_imgs = []
    B_num_imgs = 0
    B_imgs = []

    A_pool = [pool_size,A_num_imgs,A_imgs]

    B_pool = [pool_size,B_num_imgs,B_imgs]

    main_path = os.path.dirname(os.path.abspath('__file__'))

    save_model_path = main_path + '/models'
    save_image_path = main_path + '/saved_images'
    train_path = main_path+'/datasets/noise2color'

    startTraining(50, 96, 64, 2,save_model_path,save_image_path,schedulers,optimizers)
    
elif run_choice == 'classifier':

    # Create and save classifers
    main_path = os.path.dirname(os.path.abspath(__file__))

    data_path = main_path + '/datasets/faces'
    save_classifier_path = main_path + '/models/classifiers'
    createClassifier(3,data_path,save_classifier_path,64,'color')
    
elif run_choice == 'genetic algorithm':
    
    # noise2color and color2render model pathways

    main_path = os.path.dirname(os.path.abspath('__file__'))

    n2c_NN_path = main_path+'/models/noise2color/n2c '

    c2r_NN_path = main_path +'/models/color2render/c2r '

    # noise data path

    noise_data_path = main_path+'/datasets/single_noise'
    noise = get_images(noise_data_path,256,'color')

    #noise = createDataset(noise_data_path,256,256,'color')

    reupload_path = main_path+'/GA/chain'
    chain_path = main_path+'/GA/chain/trainA/1'

    save_path = main_path+'/GA/score/trainA/1'
    score_path = main_path+'/GA/score'

    discriminator_path = main_path+'/models/classifiers/d (1)'
    
    # Start Genetic Algorithm Process

    classifier = createNN('discriminator',True)
    classifier.load_state_dict(torch.load(discriminator_path,map_location = 'cuda'))

    pop,scores,pop_desc = createPopulation(noise,save_path,10,3)
    popu,best_chains,best_chain_scores = geneticAlgorithm(100,pop,scores,noise,chain_path,save_path,score_path,classifier)
    
elif run_choice == 'multi render':
    
    # Start multiple NN render sequence
    
    main_path = os.path.dirname(os.path.abspath('__file__'))
    
    model_path = main_path+'/models/noise2color/n2c ('
    noise_data_path = main_path+'/datasets/single_noise'
    save_path = main_path+'/saved_images'
    multiNNRender(noise_data_path,model_path,save_path,1,5,64)
    
elif run_choice == 'app':
    # GUI Creation -------------------------------------------

    master = Tk()

    w = Canvas(master, width=640, height=340)
    w.pack()
    image_path = ''
    NN_path = ''

    createNewImageButton = Button(master, text ="Create New Image", command = createNewImage,height = 1,padx = 5)
    importImageButton = Button(master, text ="Import Image", command = imagePath,height = 1,padx = 5)
    importNNButton = Button(master, text ="Import Neural Network", command = NNPath,height = 1,padx = 5)

    importNNButton.place(x = 110,y=10)
    importImageButton.place(x = 10,y=10)
    createNewImageButton.place(x = 350,y=10)

    w.create_rectangle(10,50,290,330, fill='tomato')
    w.create_rectangle(350,50,630,330, fill='lightblue')
    w.create_rectangle(300,180,340,190, fill='grey')
    w.create_rectangle(265,10,290,35, fill='lightgrey')
    
    # Start app

    G2,device,transform,save_path,new_path = app(False)
    mainloop()